#### Dependency and Installation

In [0]:
%run ../common/installation_setup

Catalog Name: llm_workspace
Schema Name: default
Working Directory: /Workspace/Shared/genai-databricks-hackathon-2024/databricks-notebooks/code-and-regulations
General Volume Name: /Volumes/llm_workspace/default/regubim-ai-general-volume/
install_env.sh
room-relationship-visualisation-min.html
room-route-visualisation-min.html
Regulation Volume Name: /Volumes/llm_workspace/default/regubim-ai-regulation-data-volume/
Room Design Requirements for Engineering Disciplines.pdf
SCDF Chapter 8 - Emergency Lighting & Voice Communication Systems.pdf
BIM Volume Name: /Volumes/llm_workspace/default/regubim-ai-bim-data-volume/


#### Setup Code and Regulation Rag Chain 
- Retriever
- Prompt Template
- LLM Chain

In [0]:
# Check the registered models
import mlflow

print(mlflow.__version__)

from mlflow.tracking import MlflowClient

# Initialize the MLflow client
client = MlflowClient()

# Search for all registered models
registered_models = client.search_registered_models()
embedding_model = ""
embedding_model_name = ""
embedding_model_full_name = ""
embedding_model_version = 1

# Print details of each registered model
for model in registered_models:
    if (f"{catalog_name}.{schema_name}") in model.name:

        embedding_model_full_name = model.name
        embedding_model_name = model.name.replace(f"{catalog_name}.{schema_name}.", "")
        print(f"Name: {embedding_model_name}")

        embedding_model = client.get_registered_model(name=model.name)
        versions = client.search_model_versions(filter_string=f"name='{model.name}'")

        for version in versions:
            print(f" - Version: {version.version}, Stage: {version.current_stage}")
        
        # Get the latest version    
        embedding_model_version= versions[-1].version

run_id = client.get_model_version(name=embedding_model_full_name, version=embedding_model_version).run_id

2.16.2
Name: hugging_face_sentence_transformer_model
 - Version: 1, Stage: None


In [0]:
from databricks.vector_search.client import VectorSearchClient
from langchain.embeddings import DatabricksEmbeddings
from langchain_core.runnables import RunnableLambda
from langchain.docstore.document import Document
from flashrank import Ranker, RerankRequest

from langchain.chains import LLMChain, StuffDocumentsChain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnableLambda

import mlflow.pyfunc
import pandas as pd

def create_code_regulation_rag_chain(llm_model):
    vs_endpoint_name = f"vs_endpoint_{catalog_name}"
    vs_index_fullname = f"{catalog_name}.{schema_name}.code_regulations_engineering_self_managed_vs_index"
    
    def retrieve(inputs):
        query = inputs
        vsc = VectorSearchClient(disable_notice=True)
        vs_index = vsc.get_index(endpoint_name=vs_endpoint_name, index_name=vs_index_fullname)
        loaded_embedding_model = mlflow.pyfunc.load_model(f"runs:/{run_id}/{embedding_model_name}")
        query_embeddings = loaded_embedding_model.predict(pd.DataFrame({"text": [query]}))
        query_vector = query_embeddings.iloc[0].tolist()
        retrieved = vs_index.similarity_search(
            query_vector=query_vector,
            columns=["id", "document_name", "section", "content", "url"],
            num_results=10)
        return {'query': query, 'retrieved': retrieved}

    def rerank(inputs, ranker_model="ms-marco-MiniLM-L-12-v2", k=10):
        query = inputs['query']
        retrieved = inputs['retrieved']
        passages = [
            {
                "score": doc[-1], 
                "id": doc[0], 
                "file": doc[1], 
                "section": doc[2], 
                "chunk": doc[3],
                "url": doc[4]
            } for doc in retrieved.get("result", {}).get("data_array", [])
        ]
        ranker = Ranker(model_name=ranker_model)
        rerankrequest = RerankRequest(query=query, passages=[
            {
                'id': passage['id'],
                'text': passage['chunk'],
                "meta": {
                    "section": passage['section'],
                    "document_name": passage['file'],
                    "url": passage['url']
                }
            } for passage in passages
        ])
        results = ranker.rerank(rerankrequest)[:k]
        reranked_docs = [Document(
            page_content=r.get("text"),
            metadata={
                "rank_score": r.get("score"), 
                "vector_id": r.get("id"),
                "section": r.get("meta").get("section"),
                "document_name": r.get("meta").get("document_name"),
                "url": r.get("meta").get("url")
            }
        ) for r in results]
        return {'query': query, 'retrieved_docs': reranked_docs}

    def process_retrieved_docs(inputs):
        query = inputs['query']
        docs = inputs['retrieved_docs']
        grouped_documents = {}
        metadata_section = []
        
        for doc in docs:
            key = f"{doc.metadata.get('document_name')} - {doc.metadata.get('section')}"
            if key not in grouped_documents:
                grouped_documents[key] = []
            grouped_documents[key].append(doc.page_content)
            
            if key not in metadata_section:
                metadata_info = f"Section: {key}"
                url = doc.metadata.get('url')
                if url:
                    metadata_info += f", URL: {url}"
                metadata_section.append(metadata_info)

        final_context = "\n\n".join("\n".join(pages) for pages in grouped_documents.values())
        final_metadata_section = ", ".join(metadata_section)

        return {
            "context": final_context,
            "metadata_section": final_metadata_section,
            "input": query,
            "input_documents": docs
        }

    def format_output(inputs):
        output_text = inputs.get('output_text', '')
        input_documents = inputs.get('input_documents', [])
        
        references_dict = {}
        
        for doc in input_documents:
            doc_name = doc.metadata.get('document_name', 'Unknown Document')
            full_section = doc.metadata.get('section', 'Unknown Section')
            url = doc.metadata.get('url', '')
            
            parts = full_section.split(',', 1)
            main_section = parts[0].strip()
            subsection = parts[1].strip() if len(parts) > 1 else ''
            
            if doc_name not in references_dict:
                references_dict[doc_name] = {'url': url, 'main_section': main_section, 'subsections': set()}
            
            if subsection:
                references_dict[doc_name]['subsections'].add(subsection)
        
        references = []
        for doc_name, info in references_dict.items():
            url = info['url']
            main_section = info['main_section']
            subsections = ', '.join(sorted(info['subsections']))
            
            if subsections:
                section_text = f"{main_section}, {subsections}"
            else:
                section_text = main_section
            
            if url:
                reference = f"* [{doc_name}]({url}), Section: {section_text}"
            else:
                reference = f"* {doc_name}, Section: {section_text}"
            
            references.append(reference)
        
        if references:
            references_text = "\n".join(sorted(references))
            output_text += f"\n\nTo ensure compliance, please refer to the following documents:\n\n{references_text}"
        
        return {'input': inputs.get('input'), 'output': output_text}
    
    # Define the main prompt template
    code_regulation_prompt_template = """
    You are an assistant specializing in building codes, safety regulations, and design standards for various room types in buildings. Your task is to extract and provide relevant information about codes, regulations, and standards for the room type mentioned in the question.

    Use the following pieces of context and metadata:

    <context>
    {context}
    </context>

    <metadata>
    Section: {metadata_section}
    </metadata>

    Follow these steps:
    1. Identify the room type mentioned in the question.
    2. Extract and list all relevant codes, regulations, standards, and requirements for the identified room type. Include specific measurements, materials, equipment, location, and any other pertinent details if available.
    3. Organize the information clearly, grouping related requirements together.
    4. If specific information for the mentioned room type is not available, provide general building codes or regulations that might be applicable.

    Provide only factual information from the given context. Do not make assumptions or assessments about compliance. If certain information is not available, clearly state this.

    Question: {input}

    Answer:
    """

    # Create the main prompt
    code_regulation_prompt = ChatPromptTemplate.from_template(code_regulation_prompt_template)

    # Create the document prompt
    code_regulation_prompt_document_prompt = PromptTemplate(
        input_variables=["page_content"],
        template="{page_content}"
    )

    # Create the LLM chain
    llm_chain = LLMChain(llm=llm_model, prompt=code_regulation_prompt)

    # Create the StuffDocumentsChain
    stuff_chain = StuffDocumentsChain(
        llm_chain=llm_chain,
        document_prompt=code_regulation_prompt_document_prompt,
        document_variable_name="context"
    )

    # Combine the chains
    retriever_chain = RunnableLambda(retrieve) | RunnableLambda(rerank)
    final_chain = retriever_chain | RunnableLambda(process_retrieved_docs) | stuff_chain | RunnableLambda(format_output)

    return final_chain

In [0]:
from langchain_community.chat_models import ChatDatabricks

llm_model = ChatDatabricks(endpoint="databricks-meta-llama-3-1-70b-instruct", max_tokens=3000, temperature=0.0)
code_regulation_rag_chain = create_code_regulation_rag_chain(llm_model)

### Test the code and regulation rag chain

In [0]:
# Example question to pass to your process_question function
code_regulation_question_1 = {"query": "What are the regulations for fire exits?"}

# Call the function to get the answer
code_regulation_answer_1 = code_regulation_rag_chain.invoke(code_regulation_question_1["query"])

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection


In [0]:
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

console = Console()

# Create Markdown objects with added headers
md_input_1 = Markdown(f"**Input:**\n\n{code_regulation_answer_1['input']}")
md_output_1 = Markdown(f"**Output:**\n\n{code_regulation_answer_1['output']}")

# Print input and output in panels
console.print(Panel(md_input_1, title="Input", expand=False))
console.print(Panel(md_output_1, title="Output", expand=False))

INFO:py4j.clientserver:Received command c on object id p0


╭───────────────────────────────────────────────────── Input ─────────────────────────────────────────────────────╮
│ Input:                                                                                                          │
│                                                                                                                 │
│ What are the regulations for fire exits?                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Output ─────────────────────────────────────────────────────╮
│ Output:                                                                                                         │
│                                                                                                                 │
│ Based on the provided context, the regulations for fire exits are as follows:                                   │
│                                                                                                                 │
│ Exit Lighting                                                                                                   │
│                                                                                                                 │
│  • Exits of all buildings, except for PG I, shall be provided with artificial lighting facilities to the        │
│    satisfaction of the requirements under this Code. (8.1 EXIT LIGHTING AND EXIT SIGN, 8.1.1 Exit lighting)     │
│  • The minimum illuminance to be provided for all exits and the spacing for luminaires shall be in accordance   │
│    with the requirements in SS 563. (8.1 EXIT LIGHTING AND EXIT SIGN, 8.1.1 Exit lighting)                      │
│  • The delay between the failure of the electrical supply to normal lighting and the energisation of the exit   │
│    lighting shall not exceed 1 sec. (8.1 EXIT LIGHTING AND EXIT SIGN, 8.1.1 Exit lighting)                      │
│                                                                                                                 │
│ Exit Signs                                                                                                      │
│                                                                                                                 │
│  • The entrance to every exit on every floor shall be clearly indicated by an exit sign placed over the exit    │
│    door. Such signs shall be placed so as to be clearly visible at all times. (8.1 EXIT LIGHTING AND EXIT SIGN, │
│    8.1.7 Exit and exit directional signs)                                                                       │
│  • Exit sign shall be provided over all the exit access doors for rooms with more than one door. (8.1 EXIT      │
│    LIGHTING AND EXIT SIGN, 8.1.7 Exit and exit directional signs)                                               │
│  • Exit sign shall be provided over final discharge door within exit enclosure such as exit staircase and       │
│    internal exit passageway having other exit/ exit access door opening into the exit enclosure at the          │
│    discharge floor. (8.1 EXIT LIGHTING AND EXIT SIGN, 8.1.7 Exit and exit directional signs)                    │
│                                                                                                                 │
│ Exemptions                                                                                                      │
│                                                                                                                 │
│  • Exit sign is not required under the following situations:                                                    │
│     • PG I and II buildings                                                                                     │
│     • Room provided with emergency lighting When a room is provided with only one door                          │
│     • Room without emergency lighting (i) When a room is provided with only one door and the direct distance    │
│       from the furthest distance in the room to the exit access door is 7m or less; or (ii) When the furthest   │
│       point in the room to the exit access door is 13m or less and wall of the room comprises not less than 50% │
│       clear glazing facing: an internal corridor covered by emergency lighting, an external corridor, or the    │
│       external of the building.                       

In [0]:
# Example question to pass to your process_question function
code_regulation_question_2 = {"query": "What are the regulations for fcc?"}

# Call the function to get the answer
code_regulation_answer_2 = code_regulation_rag_chain.invoke(code_regulation_question_2["query"])

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


In [0]:
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

console = Console()

# Create Markdown objects with added headers
md_input_2 = Markdown(f"**Input:**\n\n{code_regulation_answer_2['input']}")
md_output_2 = Markdown(f"**Output:**\n\n{code_regulation_answer_2['output']}")

# Print input and output in panels
console.print(Panel(md_input_2, title="Input", expand=False))
console.print(Panel(md_output_2, title="Output", expand=False))

INFO:py4j.clientserver:Received command c on object id p0


╭───────────────────────────────────────────────────── Input ─────────────────────────────────────────────────────╮
│ Input:                                                                                                          │
│                                                                                                                 │
│ What are the regulations for fcc?                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Output ─────────────────────────────────────────────────────╮
│ Output:                                                                                                         │
│                                                                                                                 │
│ Fire Command Centre (FCC) Regulations                                                                           │
│                                                                                                                 │
│ Provision                                                                                                       │
│                                                                                                                 │
│  • A Fire Command Centre (FCC) shall be provided in any building that requires a one-way or two-way emergency   │
│    voice communication system, a fire lift, or an engineered smoke control system, with exceptions for          │
│    buildings under PG I and II (PG II building having not more than two basement storeys used solely for car    │
│    parking).                                                                                                    │
│  • FCC shall be provided for all large buildings under PG III (not applicable to primary school, secondary      │
│    school, and junior colleges), IV, V, VI, VII, and VIII with AFA greater than 5000m² or having a total        │
│    occupant load exceeding 1000 persons.                                                                        │
│  • FCC shall be provided for all buildings belonging to PG III, IV, V, VI, VII, and VIII of more than 24m in    │
│    habitable height.                                                                                            │
│                                                                                                                 │
│ Size                                                                                                            │
│                                                                                                                 │
│  • An FCC shall be of adequate size to house all the terminals and supervisory/control equipment of the         │
│    building's fire protection/detection systems.                                                                │
│  • A free working space (unobstructed by door swing) of at least 6m² and minimum clear width of 2m shall be     │
│    provided.                                                                                                    │
│                                                                                                                 │
│ Location                                                                                                        │
│                                                                                                                 │
│  • The FCC shall be located at the same level as the fire engine accessway or fire engine access road.          │
│  • The FCC entrance shall be located in the following order of priority:                                        │
│     1 Within 5m from the entrance of the fire lift lobby at the designated storey of the building.              │
│     2 Within vicinity of the fire engine accessway or fire engine access road and within 5m from the entrance   │
│       of one of the protected stairs serving all storeys of the development.                                    │
│     3 At any other location as may be designated by the SCDF.                                                   │
│                                                                                                                 │
│ Construction                                                                                                    │
│                                                       

#### Create Code and Regulation Rag Model and save to Registry (Optional)

In [0]:
# # Only run this if want to create model in model registry
# from mlflow.models import infer_signature
# import mlflow
# import langchain

# code_regulation_model_name = "code_regulations_rag_model"

# # save the langchain model to registry if needed
# with mlflow.start_run(run_name="code_regulations_rag_model") as run:
#     # Infer the signature from the question and answer
#     code_regulation_signature = infer_signature(code_regulation_question, code_regulation_answer)

#     # Log the code_regulation_final_chain model to MLflow with retriever
#     model_info = mlflow.langchain.log_model(
#         code_regulation_final_chain,
#         loader_fn=code_regulation_get_retriever,  # Ensure get_retriever is defined
#         artifact_path="code_regulation_final_chain",
#         registered_model_name=code_regulation_model_name,
#         pip_requirements=[
#             "mlflow==" + mlflow.__version__,
#             "langchain==" + langchain.__version__,
#             "databricks-vectorsearch",
#         ],
#         input_example=code_regulation_question,  # Set the input example as the question
#         signature=code_regulation_signature  # Add the inferred signature
#     )